In [35]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error as mse

#from data_handler import get_sp500_ticker_list
from data_handler import get_adj_close
#from data_handler import sp500_closes_to_csv

In [5]:
# Getting the data
start='2018-01-01'
end='2020-12-31'
interval='1d'
ticker_list = 'AAPL'

df = get_adj_close(ticker_list, start, end, interval)

[*********************100%***********************]  1 of 1 completed


In [6]:
# Computing returns
ret = 100 * df.pct_change()
ret.dropna(inplace=True)

In [23]:
# Computing realized volatility
realized_vol = ret.rolling(5).std()
realized_vol = pd.DataFrame(realized_vol)
realized_vol.reset_index(drop=True, inplace=True)

In [24]:
returns_svm = ret ** 2
returns_svm = returns_svm.reset_index()
del returns_svm['Date']

In [25]:
X = pd.concat([realized_vol, returns_svm], axis=1, ignore_index=True)
X = X[4:].copy()
X = X.reset_index()
X.drop('index', axis=1, inplace=True)

In [26]:
realized_vol = realized_vol.dropna().reset_index()
realized_vol.drop('index', axis=1, inplace=True)

In [27]:
n = 252
split_date = ret.iloc[-n:].index

### Neural nets

In [29]:
NN_vol = MLPRegressor(learning_rate_init=0.001, random_state=1)

In [30]:
para_grid_NN = {'hidden_layer_sizes': [(100, 50), (50, 50), (10, 100)],
                'max_iter': [500, 1000],
                'alpha': [0.00005, 0.0005 ]}

In [31]:
clf = RandomizedSearchCV(NN_vol, para_grid_NN)
clf.fit(X.iloc[:-n].values,
        realized_vol.iloc[1:-(n-1)].values.reshape(-1, ))
NN_predictions = clf.predict(X.iloc[-n:])

In [33]:
NN_predictions = pd.DataFrame(NN_predictions)
NN_predictions.index = ret.iloc[-n:].index

In [36]:
rmse_NN = np.sqrt(mse(realized_vol.iloc[-n:] / 100, 
                      NN_predictions / 100))
print('The RMSE value of NN is {:.6f}'.format(rmse_NN))

The RMSE value of NN is 0.003438


### DL

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [39]:
model = keras.Sequential(
    [layers.Dense(256, activation="relu"),
     layers.Dense(128, activation="relu"),
     layers.Dense(1, activation="linear"),])

2022-05-01 16:05:14.711987: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [40]:
model.compile(loss='mse', optimizer='rmsprop')

In [42]:
epochs_trial = np.arange(100, 400, 4)
batch_trial = np.arange(100, 400, 4)
DL_pred = []
DL_RMSE = []
for i, j, k in zip(range(4), epochs_trial, batch_trial):
    model.fit(X.iloc[:-n].values,
        realized_vol.iloc[1:-(n-1)].values.reshape(-1,),
        batch_size=k, epochs=j, verbose=False)
    DL_predict = model.predict(np.asarray(X.iloc[-n:]))
    DL_RMSE.append(np.sqrt(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100)))
    DL_pred.append(DL_predict)
    print('DL_RMSE_{}:{:.6f}'.format(i+1, DL_RMSE[i]))

DL_RMSE_1:0.004491
DL_RMSE_2:0.004243
DL_RMSE_3:0.006194
DL_RMSE_4:0.003906


In [43]:
DL_predict = pd.DataFrame(DL_pred[DL_RMSE.index(min(DL_RMSE))])
DL_predict.index = ret.iloc[-n:].index

In [15]:
len(realized_vol.iloc[:-n].values)

498

In [18]:
realized_vol.iloc[1:-(n-1)].values#.reshape(-1, )

array([0.58395365, 0.59564529, 0.55725517, 0.59667709, 0.85071099,
       0.83330167, 0.94839978, 0.98265876, 0.94274191, 0.69044052,
       0.76229491, 0.91433315, 1.08086235, 0.95840225, 1.101341  ,
       1.00653947, 1.94245531, 1.99355576, 3.22914194, 3.27672485,
       3.28963124, 3.0235482 , 3.28715126, 2.76168624, 2.44852167,
       1.33766815, 1.75890057, 1.56127793, 1.71439933, 1.62024043,
       0.96441178, 1.13151617, 1.12856682, 1.05315955, 1.5561496 ,
       1.37335703, 0.93687902, 0.93748619, 0.99225903, 0.77599471,
       1.02633845, 1.05112429, 1.24056231, 1.22033543, 1.15701901,
       0.78885325, 0.62554551, 0.66988818, 1.03745291, 0.9126849 ,
       0.92295894, 2.94532854, 3.11033848, 3.01185742, 3.01088423,
       2.78752729, 1.4679686 , 1.24523304, 0.92288336, 1.74124155,
       1.72214868, 1.83699344, 1.71087674, 1.74061545, 0.8727051 ,
       0.86226306, 0.69928761, 0.61198242, 1.61064895, 2.33650975,
       2.20687505, 1.67863316, 1.84777846, 1.8795149 , 0.84067